In [69]:
import numpy as np  # arrays
import matplotlib.pyplot as plt  # 2d plotting
from numpy.linalg import norm
from scipy.optimize import minimize_scalar #Exact line search algorithm

In [70]:
a,b = 1,100    # parameters for Rosenbrock function
f = lambda x,y: (a-x)**2+b*(y-x**2)**2
Df = lambda x,y: np.array([2*(x-a)-4*b*x*(y-x**2),
                           2*b*(y-x**2)])
D2f = lambda x,y: np.array([[2-4*b*y+12*b*x**2,-4*b*x],
                            [-4*b*x,2*b]])

In [71]:
def objFunc(x): # Rosenbrock when the input is in vector form
    return 100*(x[1] - x[0]**2)**2 + (1 - x[0])**2
def GradObjFunc(x): # gradient of Rosenbrock
    return np.array([400*x[0]**3-400*x[0]*x[1]+2*x[0]-2, -200*x[0]**2+200*x[1]])

In [72]:
def PR_beta(Dfk,Dfk1, pk): #doesn't use pk, just including the parameter since other beta methods use pk
    return ((Dfk1-Dfk)@Dfk1) / (Dfk@Dfk)
def FR_beta(Dfk,Dfk1, pk): #doesn't use pk
    return (Dfk1@Dfk1) / (Dfk@Dfk)

In [74]:
def nonlinearCG(x0, y0, f, Df, beta, restart=True, tol=1e-8, max_steps=10000):
    x,y = x0,y0
    path = [[x,y]]
    i=0                   # iteration count
    dx = Df(x,y)          # current gradient
    pk = -dx
    while np.linalg.norm(dx)>tol and i<max_steps:    
        def subproblem1D(alpha):
          return objFunc([x,y] + alpha * pk)
        res = minimize_scalar(subproblem1D) # scipy function to minimize objFunction w.r.t alpha
        alpha = res.x

        xnew,ynew = x + alpha*pk[0], y + alpha*pk[1]
        dx1 = Df(xnew,ynew)      # Df_{k+1}
        bk = beta(dx,dx1,pk)  # beta_k
        if restart and i%3 == 0: #restart every 3 iterations
            bk=0
        pk = -dx1 + bk*pk
        path.append([xnew,ynew])
        x,y = xnew,ynew
        dx = dx1
        i += 1
    return np.array(path), i, x, y

## Part A

In [75]:
x0,y0 = 1.2,1.2
path_PR, i, x, y = nonlinearCG(x0, y0, f, Df, PR_beta, False)
print(f'After {i} iterations, approximate minimum is {f(x,y)} at {x,y}')

After 10 iterations, approximate minimum is 4.930380657631324e-29 at (1.0000000000000067, 1.0000000000000135)


In [76]:
minimizer = [1,1]
L = norm(path_PR[-1]-minimizer)/norm(path_PR[-2]-minimizer)
print('L equals to ',L)

L equals to  5.2523719516402525e-05


In [77]:
path_FR, i, x, y=nonlinearCG(x0, y0, f, Df, FR_beta, False)
print(f'After {i} iterations, approximate minimum is {f(x,y)} at {x,y}')

After 30 iterations, approximate minimum is 3.7186327915733286e-17 at (1.0000000060888288, 1.0000000122111974)


In [78]:
L = norm(path_FR[-1]-minimizer)/norm(path_FR[-2]-minimizer)
print('L equals to ',L)

L equals to  1.0035993815083593


Without the restart algorithm, PR method converges super-linearly(L is close to 0) and FR method converges linearly (L is close to 1).

## Part B

In [79]:
path_PR, i, x, y = nonlinearCG(x0, y0, f, Df, PR_beta, True)
print(f'After {i} iterations, approximate minimum is {f(x,y)} at {x,y}')

After 13 iterations, approximate minimum is 4.738095811983702e-29 at (1.0000000000000069, 1.0000000000000138)


In [80]:
L = norm(path_PR[-1]-minimizer)/norm(path_PR[-2]-minimizer)
print('L equals to ',L)

L equals to  0.0002777279224060647


In [81]:
path_FR, i, x, y = nonlinearCG(x0, y0, f, Df, FR_beta, True)
print(f'After {i} iterations, approximate minimum is {f(x,y)} at {x,y}')

After 10 iterations, approximate minimum is 6.296760414157414e-21 at (1.000000000079289, 1.0000000001588945)


In [82]:
L = norm(path_FR[-1]-minimizer)/norm(path_FR[-2]-minimizer)
print('L equals to ',L)

L equals to  0.3844671324519976


With the restart algorithm, PR method converges super-linearly(L is close to 0) and FR method converges linearly. However, we see improvement in the FR method as it only takes 10 iterations rather than 30.

## Part C

In [83]:
x0,y0 = -1.2,1

In [84]:
path_PR, i, x, y = nonlinearCG(x0, y0, f, Df, PR_beta, False)
print(f'After {i} iterations, approximate minimum is {f(x,y)} at {x,y}')
L = norm(path_PR[-1]-minimizer)/norm(path_PR[-2]-minimizer)
print('L equals to ',L)

After 18 iterations, approximate minimum is 4.6340944995991903e-20 at (0.999999999954214, 0.9999999999294623)
L equals to  3.0089644629520566e-05


In [85]:
path_FR, i, x, y = nonlinearCG(x0, y0, f, Df, FR_beta, False)
print(f'After {i} iterations, approximate minimum is {f(x,y)} at {x,y}')
L = norm(path_FR[-1]-minimizer)/norm(path_FR[-2]-minimizer)
print('L equals to ',L)

After 217 iterations, approximate minimum is 1.7256039452241e-17 at (1.0000000041539983, 1.0000000083061582)
L equals to  0.9949263127723659


Without the restart algorithm, PR method converges super-linearly(L is close to 0) and FR method converges linearly (L is close to 1). This is similar to part a. However, changing the starting point here (farther away from the minimizer) means that the FR method takes much more iterations.

In [86]:
path_PR, i, x, y = nonlinearCG(x0, y0, f, Df, PR_beta, True)
print(f'After {i} iterations, approximate minimum is {f(x,y)} at {x,y}')
L = norm(path_PR[-1]-minimizer)/norm(path_PR[-2]-minimizer)
print('L equals to ',L)

After 27 iterations, approximate minimum is 2.4373769022542323e-22 at (0.9999999999984763, 0.9999999999985064)
L equals to  1.546477137835673e-05


In [87]:
path_FR, i, x, y = nonlinearCG(x0, y0, f, Df, FR_beta, True)
print(f'After {i} iterations, approximate minimum is {f(x,y)} at {x,y}')
L = norm(path_FR[-1]-minimizer)/norm(path_FR[-2]-minimizer)
print('L equals to ',L)

After 19 iterations, approximate minimum is 3.2868744030716063e-17 at (1.0000000057258882, 1.0000000114805772)
L equals to  0.09935116422895718


With the restart algorithm, PR method and FR method both converge super-linearly since both Ls approach 0.